# Financial Portfolio Manager

## Objective

Build a Financial Portfolio Manager that assists users in managing their investments using **Group Chat** for agent collaboration and **StateFlow** for dynamic workflow management based on user inputs.

This agentic system will begin with a **User Proxy Agent** which will initiate a chat with the **Group Chat Manager**.

The Group Chat Manager will then call the **Portfolio Analysis Agent**, which will take user inputs and summarize their investment portfolio.

Based on the insights from the Portfolio Analysis Agent, the workflow will be dynamically managed using StateFlow to either the **Growth Investment Agent** or the **Value Investment Agent**.

Finally, these agents will share their outputs with the **Investment Advisor Agent**, which will generate a personalized financial report for the user.

---

## Key Concepts to Use

| Concept | Description |
|---------|-------------|
| **Group Chat for Collaboration** | Agents will collaborate through a group chat managed by the Group Chat Manager |
| **Dynamic Workflow Management** | StateFlow will manage the workflow based on user inputs and agent recommendations |
| **Portfolio Analysis** | The Portfolio Analysis Agent will summarize the user's investment portfolio and determine the investment category (Growth or Value) |
| **Investment Recommendations** | The Growth Investment Agent and Value Investment Agent will provide specific investment suggestions |
| **Personalized Financial Report** | The Investment Advisor Agent will compile a detailed report based on the insights from previous agents |

---

## Expected Conversation Flow

### Step 1: User Proxy Agent

- **Role:** Initiates the chat and activates the Group Chat Manager
- **Action:** Informs the Group Chat Manager about the user's intention to manage their investments

---

### Step 2: Group Chat Manager

- **Role:** Manages the group chat and directs the conversation flow
- **Action:** Calls the Portfolio Analysis Agent to take user inputs

---

### Step 3: Portfolio Analysis Agent

- **Role:** Summarizes the user's existing portfolio and determines the investment category

- **Inputs Expected:**
  - Current salary
  - Summary of current investment portfolio (e.g., amounts in fixed deposits, SIPs, real estate, etc.)

- **Output:** Determines whether to pursue Growth or Value investments based on the user's portfolio

---

### Step 4: StateFlow Workflow Management

- **Role:** Dynamically manages the workflow based on the Portfolio Analysis Agent's recommendation
- **Action:** Directs the workflow to either the Growth Investment Agent or the Value Investment Agent

---

### Step 5: Growth Investment Agent *or* Value Investment Agent

#### Growth Investment Agent
- **Role:** Suggests investments for maximizing portfolio growth
- **Action:** Provides recommendations for high-growth investment options

#### Value Investment Agent
- **Role:** Recommends stable investments for long-term value
- **Action:** Provides recommendations for stable, long-term investment options

---

### Step 6: Investment Advisor Agent

- **Role:** Compiles a detailed report of holdings and recommendations
- **Action:** Generates a personalized financial report based on the insights from previous agents

---

## Expected Output

A personalized financial report including portfolio analysis, growth, and value investment suggestions.

> **Note:** Display the output of the Investment Advisor Agent.

In [1]:
# Import the libraries
import autogen
from autogen import ConversableAgent, GroupChat, GroupChatManager
from IPython.display import Markdown, display

In [2]:
# load env variables
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# configuration for LLM
config_list = {
    "config_list": [{"model": "gpt-4o-mini", "temperature": 0.1}]
}

In [8]:
portfolio_analyst_agent = ConversableAgent(
    "portfolio_analyst_agent",
    system_message="""You are a financial analyst. user will provide their current salary and summary of current investments portfolio.
    You will analyze the existing portfolio and determines the investment category. From your analysis determine 
    whether to pursue Growth or Value investments based on the user's portfolio. End the response with the investment category.
    """,
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [9]:
user_agent = ConversableAgent(
    "user_agent",
    system_message="""You are the user who want to get some solid investment advice from the financial analyst.
    Provide your current salary and portfolio details.""",
    llm_config=config_list,
    human_input_mode="NEVER",  
)

In [ ]:
growth_investment_agent = ConversableAgent(
    "growth_investment_agent",
    system_message="""You are an growth investment expert who Suggests investments for maximizing portfolio growth by recommending recommendation for high-growth investments options.""",
    llm_config=config_list,
    human_input_mode="NEVER", 
)

value_investment_agent = ConversableAgent(
    "value_investment_agent",
    system_message="""You are an value investment expert who Suggests recommendations for stable investments options for long-term investment growth.""",
     
)

In [33]:
investment_advisor_agent = ConversableAgent(
    name="investment_advisor_agent",
    system_message="""You are a financial investment advisor with expertise in portfolio management and investment strategy. 
    Generates a personalized financial report based on the insights from previous agents. The report should include detailed report of holdings and recommendations""",
    llm_config=config_list,
    human_input_mode="NEVER", 
)

In [34]:
def state_transition(last_speaker, groupchat):
    
    messages = groupchat.messages

    if last_speaker is portfolio_analyst_agent:
        if 'Investment Category: Growth' in messages[-1]['content']:
            return growth_investment_agent
        elif 'Investment Category: Value' in messages[-1]['content']:
            return value_investment_agent
    elif last_speaker is growth_investment_agent:
        return investment_advisor_agent
    elif last_speaker is value_investment_agent:
        return investment_advisor_agent
    elif last_speaker is investment_advisor_agent:
        return None

In [39]:
group_chat = GroupChat(
    agents=[portfolio_analyst_agent,growth_investment_agent, value_investment_agent,investment_advisor_agent],
    messages=[],
    max_round= 4,
    speaker_selection_method='auto',
    select_speaker_auto_llm_config=config_list
)

In [40]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
)

In [41]:
portfolio_details = {
    "salary": 100000,
    "portfolio": {
        "stocks": 50000,
        "bonds": 30000,
        "real_estate": 20000
    }
}

content = f"""
User's current salary: ${portfolio_details["salary"]}
User's current portfolio:
- Stocks: ${portfolio_details["portfolio"]["stocks"]}
- Bonds: ${portfolio_details["portfolio"]["bonds"]}
"""

In [42]:
chat_result = user_agent.initiate_chat(
    group_chat_manager,
    message=content,
    summary_method="last_msg",
)

user_agent (to chat_manager):


User's current salary: $100000
User's current portfolio:
- Stocks: $50000
- Bonds: $30000


--------------------------------------------------------------------------------

Next speaker: portfolio_analyst_agent

portfolio_analyst_agent (to chat_manager):

Based on your current salary of $100,000 and your investment portfolio, which consists of $50,000 in stocks and $30,000 in bonds, we can analyze your existing investments.

1. **Stocks ($50,000)**: This portion of your portfolio indicates a willingness to take on some risk for potential higher returns. Stocks are generally considered growth investments, especially if they are in sectors that are expected to grow rapidly.

2. **Bonds ($30,000)**: Bonds typically provide more stability and income, but they are generally considered value investments. They are less volatile than stocks and provide fixed interest payments.

Given that your portfolio has a significant allocation to stocks, which are typicall

In [45]:
agent_name_to_find = "investment_advisor_agent"

# Iterate through the dictionary to find the last conversation for the specified agent
for agent, interactions in group_chat_manager.chat_messages.items():
    # Filter interactions by agent name
    filtered_interactions = [i for i in interactions if i.get("name") == agent_name_to_find]
    if filtered_interactions:
        # Print the last conversation's content
        # print(filtered_interactions[-1]["content"])
        display(Markdown(filtered_interactions[-1]["content"]))
        break
else:
    print("Agent not found or no interactions available.")


### Personalized Financial Report

**Client Overview:**
- **Current Salary:** $100,000
- **Current Portfolio:**
  - **Stocks:** $50,000
  - **Bonds:** $30,000
- **Total Portfolio Value:** $80,000

---

### Portfolio Analysis

Your current portfolio reflects a moderate risk tolerance with a significant allocation to stocks (62.5%) and a stable component in bonds (37.5%). This balance suggests a growth-oriented strategy, but there is room for optimization to enhance potential returns.

### Current Holdings Breakdown
- **Stocks ($50,000)**: Focused on growth, but could benefit from sector diversification.
- **Bonds ($30,000)**: Provides stability but may limit overall growth potential.

---

### Recommendations for Portfolio Growth

#### 1. **Increase Stock Allocation**
   - **Target Allocation:** Consider increasing your stock holdings to approximately $60,000 by reallocating $10,000 from bonds.
   - **Sectors to Explore:**
     - **Technology**: Invest in high-growth tech stocks or ETFs (e.g., NVIDIA, Amazon, Microsoft).
     - **Healthcare/Biotech**: Look into innovative companies (e.g., Moderna, Teladoc).
     - **Renewable Energy**: Consider stocks or ETFs focused on sustainability (e.g., Tesla, NextEra Energy).

#### 2. **Diversify with Growth ETFs**
   - **Recommended ETFs**: 
     - **Invesco QQQ Trust**: Tracks the Nasdaq-100, focusing on tech-heavy growth stocks.
     - **Vanguard Growth ETF**: Provides exposure to a diversified portfolio of growth stocks.

#### 3. **Consider International Markets**
   - **Emerging Markets**: Allocate a portion to mutual funds or ETFs targeting high-growth economies (e.g., India, Brazil).

#### 4. **Real Estate Investment Trusts (REITs)**
   - **Growth REITs**: Invest in sectors like data centers and logistics for capital appreciation and income.

#### 5. **Cryptocurrencies**
   - **Digital Assets**: If comfortable with high volatility, consider allocating a small percentage (e.g., 5%) to cryptocurrencies like Bitcoin or Ethereum.

#### 6. **Venture Capital or Private Equity**
   - **Alternative Investments**: Explore opportunities in venture capital funds or private equity for potential high returns.

#### 7. **Dollar-Cost Averaging**
   - **Investment Strategy**: Implement a dollar-cost averaging approach to mitigate market volatility by investing a fixed amount regularly.

---

### Portfolio Adjustment Recommendation
- **Rebalance Your Portfolio**: 
  - **Current Allocation**: 
    - Stocks: $50,000 (62.5%)
    - Bonds: $30,000 (37.5%)
  - **Proposed Allocation**: 
    - Stocks: $60,000 (75%)
    - Bonds: $20,000 (25%)

This adjustment increases your exposure to growth while still maintaining a safety net through bonds.

---

### Conclusion
By focusing on high-growth sectors, diversifying your investments, and rebalancing your portfolio, you can maximize your growth potential. Always consider your risk tolerance and investment horizon before making significant changes. Consulting with a financial advisor can provide personalized guidance tailored to your financial goals.

**Next Steps:**
- Review the recommended sectors and investment options.
- Consider reallocating your portfolio as suggested.
- Monitor your investments regularly and adjust as needed based on market conditions and personal financial goals.